# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
#
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### QUERY 1

This query returns the song data (artist, song, length, session id and item in session id) listened by session id.

In [8]:
## Create table for query 1
query = "CREATE TABLE IF NOT EXISTS music_by_session"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Insert data   
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

**Verification:** Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [12]:
## Verify table insertion
query = "SELECT sessionId, itemInSession, artist, song, length FROM music_by_session WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute( query  )
except Exception as e:
    print( e )
print( 'sessionId\titemInSession\tartist\tsong\tlength' )
for row in rows:
    print( "{}\t{}\t{}\t{}\t{}".format(row.sessionid, row.iteminsession, row.artist, row.song, row.length) )

sessionId	itemInSession	artist	song	length
338	4	Faithless	Music Matters (Mark Knight Dub)	495.30731201171875


In [13]:
## Check against CSV
df = pd.read_csv("event_datafile_new.csv")
df.loc[(df.sessionId == 338) & (df.itemInSession == 4), ['sessionId', 'itemInSession', 'artist', 'song', 'length'] ]

,sessionId,itemInSession,artist,song,length
444,338,4,Faithless,Music Matters (Mark Knight Dub),495.3073


#### QUERY 2

This query returns the song and artist names listened by a user by session id. User name and userid are provided as well.


In [14]:
## Create table 
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userid, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## Insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
     

**Verification:** Return the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
query = "SELECT userId, sessionId, itemInSession, artist, song, firstName, lastName FROM user_session WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute( query  )
except Exception as e:
    print( e )
print("userId\tsessionId\titemInSession\tartist\tsong\tfirstName\tlastName")    
for row in rows:
    print( "{}\t{}\t{}\t{}\t{}\t{}\t{}\t".format(row.userid, row.sessionid, row.iteminsession, row.artist, row.song, row.firstname, row.lastname) )

userId	sessionId	itemInSession	artist	song	firstName	lastName
10	182	0	Down To The Bone	Keep On Keepin' On	Sylvie	Cruz	
10	182	1	Three Drives	Greece 2000	Sylvie	Cruz	
10	182	2	Sebastien Tellier	Kilometer	Sylvie	Cruz	
10	182	3	Lonnie Gordon	Catch You Baby (Steve Pitron & Max Sanna Radio Edit)	Sylvie	Cruz	


In [17]:
df.loc[(df.userId == 10) & (df.sessionId == 182), ['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName']]

,userId,sessionId,itemInSession,artist,song,firstName,lastName
4704,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
4705,10,182,1,Three Drives,Greece 2000,Sylvie,Cruz
4706,10,182,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
4707,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### QUERY 3

This query returns all the users that listened a given song.

In [18]:
## Create table 
query = "CREATE TABLE IF NOT EXISTS played_songs "
query = query + "(song text, userId int, artist text, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

## Insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO played_songs (song, userId, artist, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[0], line[1], line[4]))
        


**Verification:** Return the every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



In [19]:
query = "SELECT song, userId, artist, firstName, lastName FROM played_songs WHERE song='All Hands Against His Own'"
try:
    rows = session.execute( query  )
except Exception as e:
    print( e )

print('song\tfirstname\tlastname')
for row in rows:
    print( "{}\t{}\t{}\t".format(row.song, row.firstname, row.lastname) )

song	firstname	lastname
All Hands Against His Own	Jacqueline	Lynch	
All Hands Against His Own	Tegan	Levine	
All Hands Against His Own	Sara	Johnson	


In [20]:
df.loc[df.song == 'All Hands Against His Own', ['song', 'firstName', 'lastName']]

,song,firstName,lastName
2792,All Hands Against His Own,Tegan,Levine
5135,All Hands Against His Own,Sara,Johnson
6298,All Hands Against His Own,Jacqueline,Lynch


### Drop the tables before closing out the sessions

In [21]:
## Drop the table before closing out the sessions
query = "DROP TABLE music_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE played_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()